In [82]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from sklearn.preprocessing import Binarizer
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import train_test_split


In [74]:
data=pd.read_csv('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.csv',thousands=',')
display(data.head())

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Unnamed: 13
0,Adams Village,1861.0,0.0,0.0,NaN,0.0,0.0,0.0,12.0,2.0,10.0,0.0,0.0,NaN
1,Addison Town and Village,2577.0,3.0,0.0,NaN,0.0,0.0,3.0,24.0,3.0,20.0,1.0,0.0,NaN
2,Akron Village,2846.0,3.0,0.0,NaN,0.0,0.0,3.0,16.0,1.0,15.0,0.0,0.0,NaN
3,Albany,97956.0,791.0,8.0,NaN,30.0,227.0,526.0,4090.0,705.0,3243.0,142.0,NaN,NaN
4,Albion Village,6388.0,23.0,0.0,NaN,3.0,4.0,16.0,223.0,53.0,165.0,5.0,NaN,NaN


In [75]:
data.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3', 'Unnamed: 13'],
      dtype='object')

In [76]:
rename_dict = {
    'Property\ncrime': 'property_crime',
    'Population': 'population',
    'Murder and\nnonnegligent\nmanslaughter': 'murder_manslaughter',
    'Robbery': 'robbery',
    'Burglary':'burglary',
    'Violent\ncrime':'violent_crime',
    'Rape\n(revised\ndefinition)1':'rape',
    'Rape\n(legacy\ndefinition)2':'rape2',
    'Aggravated\nassult':'aggravated_assult',
    'Larceny-\ntheft':'larceny_theft',
    'Motor\nvehicle\ntheft':'motor_vehicle_theft',
    'Arson3':'arson',
    'Unnamed:13':'unnamed',
}
data_renamed = data.rename(rename_dict, axis='columns')


In [77]:
data_renamed.violent_crime.astype(np.float)

0         0.0
1         3.0
2         3.0
3       791.0
4        23.0
5         5.0
6         3.0
7       107.0
8         9.0
9        30.0
10        0.0
11        5.0
12        0.0
13        2.0
14       96.0
15        1.0
16        5.0
17        8.0
18       57.0
19       15.0
20        5.0
21       13.0
22      284.0
23        0.0
24        8.0
25        1.0
26        2.0
27        1.0
28        4.0
29        0.0
        ...  
321     361.0
322      16.0
323      18.0
324      40.0
325       6.0
326       6.0
327       2.0
328       4.0
329      11.0
330     109.0
331      26.0
332       6.0
333       5.0
334      21.0
335       0.0
336      11.0
337       2.0
338      35.0
339       6.0
340      78.0
341       4.0
342       0.0
343       3.0
344       7.0
345       2.0
346    1036.0
347      15.0
348       NaN
349       NaN
350       NaN
Name: violent_crime, Length: 351, dtype: float64

In [78]:
data_renamed.describe()

,population,violent_crime,murder_manslaughter,rape,rape2,robbery,Aggravated assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson,Unnamed: 13
count,3.480000e+02,348.000000,348.000000,0.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,187.000000,0.0
mean,4.003763e+04,201.594828,1.566092,NaN,5.864943,72.902299,121.261494,792.606322,119.683908,637.017241,35.905172,1.871658,NaN
std,4.500374e+05,2815.268504,18.303673,NaN,60.425452,1031.032873,1706.131730,7659.724746,924.948789,6346.054451,403.423826,10.693411,NaN
min,5.260000e+02,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,3.003000e+03,2.000000,0.000000,NaN,0.000000,0.000000,1.000000,40.500000,6.000000,31.000000,0.000000,0.000000,NaN
50%,7.233500e+03,6.000000,0.000000,NaN,0.000000,1.000000,4.000000,112.500000,17.500000,94.000000,2.000000,0.000000,NaN
75%,1.842750e+04,22.000000,0.000000,NaN,2.000000,5.000000,14.000000,341.000000,51.250000,287.250000,7.000000,1.000000,NaN
max,8.396126e+06,52384.000000,335.000000,NaN,1112.000000,19170.000000,31767.000000,141971.000000,16606.000000,117931.000000,7434.000000,132.000000,NaN


In [89]:
y=data_renamed.property_crime
X-data_renamed[['population','murder_manslaughter','robbery']]
X.murder_manslaughter[X.murder_manslaughter>0]=1
X.robbery[X.robbery>0]=1
(X_train, X_test, y_train, y_test)=train_test_split(X,y,test_size=0.25)
lr=LinearRegression()

/Users/amyscott/anaconda2/envs/nthinkfulpython/lib/python3.7/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/amyscott/anaconda2/envs/nthinkfulpython/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [90]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,normalize=False)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [91]:
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
lr.score(X_test,y_pred)
mean_squared_error(y_test,y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [48]:
X.dtypes

population_floats       object
robbery                 object
murder_manslaughter    float64
dtype: object